# Share Tensor
### With Duet Clients

## Import libs

In [1]:
import syft as sy
import torch

from sympc.session import Session

# Import our ShareTensor ControlCenter
from sympc.tensor import ShareTensorCC

## Create the clients

In [2]:
alice = sy.VirtualMachine(name="alice")
bob = sy.VirtualMachine(name="bob")

alice_client = alice.get_client()
bob_client = bob.get_client()

## Create a session

The session is used to send some config information only once between the parties.
This information can be:
* the ring size in which we do the computation
* the precision and base
* the approximation methods we are using for different functions (TODO)

In [3]:
session = Session(parties=[alice_client, bob_client])
print(session)

## Send the session to all the parties

In [4]:
Session.setup_mpc(session)

In [5]:
x = ShareTensorCC(secret=torch.tensor([1,2,3,4]), session=session)

In [6]:
print(x)

[ShareTensorCC]
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer


In [7]:
# The values does not make sense for any user, unless they "reconstruct" the original tensor
for val in x.share_ptr:
    print(val.get()) ## Attention! get() destroys the pointer

[ShareTensor]
	| [FixedPointEncoder]: precision: 4, base: 10
	| Data: tensor([-3254347133205610703,  5629128132088023381,  2996710540977179804,
         7844885257668983457])
[ShareTensor]
	| [FixedPointEncoder]: precision: 4, base: 10
	| Data: tensor([ 3254347133205620703, -5629128132088003381, -2996710540977149804,
        -7844885257668943457])


In [8]:
x_secret = torch.tensor([1,2,3,4])
x = ShareTensorCC(secret=x_secret, session=session)

In [9]:
print(x.reconstruct()) ## Attention! Also reconstruct, destroys the pointers

tensor([1., 2., 3., 4.])


## Secret is shared by the orchestrator
- the orchestrator generates shares locally
- distributes them to the parties

### Public Operations

In [10]:
x_secret = torch.tensor([1,2,3,4])
x = ShareTensorCC(secret=x_secret, session=session)

y = 50

In [11]:
print("[Priv + Pub] X + Y =", (x + y).reconstruct())
print("[Pub + Pub] X + Y =", x_secret + y)

[Priv + Pub] X + Y = tensor([51., 52., 53., 54.])
[Pub + Pub] X + Y = tensor([51, 52, 53, 54])


In [12]:
print("[Priv - Pub] X - Y =", (x - y).reconstruct())
print("[Pub - Pub] X - Y =", x_secret - y)

[Priv - Pub] X - Y = tensor([-49., -48., -47., -46.])
[Pub - Pub] X - Y = tensor([-49, -48, -47, -46])


In [13]:
print("[Priv * Pub] X * Y =", (x * y).reconstruct())
print("[Pub * Pub] X * Y =", x_secret * y)

[Priv * Pub] X * Y = tensor([ 50., 100., 150., 200.])
[Pub * Pub] X * Y = tensor([ 50, 100, 150, 200])


### Private Operations

In [14]:
x_secret = torch.tensor([1, 2, 3])
x = ShareTensorCC(secret=x_secret,session=session)

y_secret = 50
y = ShareTensorCC(secret=y_secret, session=session)

In [15]:
print("[Priv + Priv] X + Y =", (x + y).reconstruct())
print("[Pub + Pub] X + Y =", x_secret + y_secret)

[Priv + Priv] X + Y = tensor([51., 52., 53.])
[Pub + Pub] X + Y = tensor([51, 52, 53])


In [16]:
print("[Priv - Priv] X - Y =", (x - y).reconstruct())
print("[Pub - Pub] X - Y =", x_secret - y_secret)

[Priv - Priv] X - Y = tensor([-49., -48., -47.])
[Pub - Pub] X - Y = tensor([-49, -48, -47])


In [17]:
print("[Priv * Priv] X * Y =", (x * y).reconstruct())
print("[Pub * Pub] X * Y =", x_secret * y_secret)

[Priv * Priv] X * Y = tensor([ 50., 100., 150.])
[Pub * Pub] X * Y = tensor([ 50, 100, 150])


## Private Secret - owned by one party
- are used more generators to construct PRZS (Pseudo-Random-Zero-Shares)
- the party that has the secret would add it to their own share

### Public Operations

In [18]:
x_secret = alice_client.torch.Tensor([1,2,3,4]) # at Alice
x = ShareTensorCC(secret=x_secret, shape=(1, 4), session=session)
y = 50

In [19]:
print("[Priv + Pub] X + Y =", (x + y).reconstruct())
print("[Pub + Pub] X + Y =", (x_secret + y).get()) # On Alice side

[Priv + Pub] X + Y = tensor([[51., 52., 53., 54.]])
[Pub + Pub] X + Y = tensor([51., 52., 53., 54.])


In [20]:
print("[Priv - Pub] X - Y =", (x - y).reconstruct())
print("[Pub - Pub] X - Y =", (x_secret - y).get()) # On Alice side

[Priv - Pub] X - Y = tensor([[-49., -48., -47., -46.]])
[Pub - Pub] X - Y = tensor([-49., -48., -47., -46.])


In [21]:
print("[Priv * Pub] X * Y =", (x * y).reconstruct())
print("[Pub * Pub] X * Y =", (x_secret * y).get()) # On Alice side

[Priv * Pub] X * Y = tensor([[ 50., 100., 150., 200.]])
[Pub * Pub] X * Y = tensor([ 50., 100., 150., 200.])


### Private Operations

In [22]:
x_secret = alice_client.torch.Tensor([1,2,3]) # at Alice
x = ShareTensorCC(secret=x_secret, shape=(1,3), session=session)

y_secret = bob_client.python.Int(50) # At Bob
y = ShareTensorCC(secret=y_secret, shape=(1,), session=session)

In [23]:
print("[Priv + Priv] X + Y =", (x + y).reconstruct())
# We can simply add them because they are at different locations
# That is why we need to first get them before adding them
print("[Pub + Pub] X + Y =", x_secret.get_copy() + y_secret.get_copy())

[Priv + Priv] X + Y = tensor([[51., 52., 53.]])
[Pub + Pub] X + Y = tensor([51., 52., 53.])


In [24]:
print("[Priv - Priv] X - Y =", (x - y).reconstruct())
print("[Pub - Pub] X - Y =", x_secret.get_copy() - y_secret.get_copy())

[Priv - Priv] X - Y = tensor([[-49., -48., -47.]])
[Pub - Pub] X - Y = tensor([-49., -48., -47.])


In [25]:
print("[Priv * Priv] X * Y =", (x * y).reconstruct())
print("[Pub * Pub] X * Y =", x_secret.get_copy() * y_secret.get_copy())

[Priv * Priv] X * Y = tensor([[ 50., 100., 150.]])
[Pub * Pub] X * Y = tensor([ 50., 100., 150.])
